In [11]:
import imageio
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [16]:
def sobel_filtering(im):
    dx = ndimage.sobel(im, 0)  # horizontal derivative
    dy = ndimage.sobel(im, 1)  # vertical derivative
    mag = np.hypot(dx, dy)  # magnitude
    mag *= 255.0 / np.max(mag)  # normalize (Q&D)
    return mag

In [17]:
def morf_operations(img_in,kernel):
    dilation = cv2.dilate(img_in,kernel,iterations = 1)
    closing = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel)
    erosion = cv2.erode(dilation,kernel,iterations = 1)
    return erosion

In [18]:
# Cargar imagen
video = imageio.mimread('1472 semen-00.avi')

In [19]:
# Deteccion img grises

img_in = video[0]

# Aplicar LoG
greyscale = cv2.cvtColor(img_in, cv2.COLOR_BGR2GRAY) # Convertir a grises
gaussian_blur = cv2.GaussianBlur(greyscale,(3,3),1.6)
laplacian = cv2.Laplacian(gaussian_blur,cv2.CV_64F)

positive_laplacian = np.abs(np.min(laplacian))+laplacian

In [20]:
# Otsu
img_inv_uint8 = np.array(positive_laplacian, dtype=np.uint8)
tresh,otsu = cv2.threshold(img_inv_uint8,0,np.max(img_inv_uint8).astype(int),cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# Binarizar
img_out = np.where(otsu==np.max(img_inv_uint8).astype(int),0,1)

In [10]:
# Visualizaciones

%matplotlib qt

imagenes = [greyscale, gaussian_blur, laplacian, otsu, img_out]
nombres = ['greyscale', 'gaussiano', 'laplaciano', 'otsu', 'binary']
for i in range(1,6):
    plt.subplot(2,3,i),plt.title(nombres[i-1]),plt.imshow(imagenes[i-1][0:170,0:170],cmap='gray')

In [21]:
# Hallar puntos de las particulas

from skimage.measure import label
labeled_img, total_particles = label(img_out, connectivity=2, return_num=True)

In [22]:
# Crear dataframe

import pandas as pd
particles = pd.DataFrame(index=range(total_particles), columns=['id', 'x', 'y', 'total_pixels'])


for p in range(1, total_particles+1):
    particles.loc[p - 1, ['id']] = p
    coords_p = np.argwhere(labeled_img == p)
    particles.loc[p-1, ['x']] = np.mean(coords_p[:, 0])
    particles.loc[p-1, ['y']] = np.mean(coords_p[:, 1])
    particles.loc[p-1, ['total_pixels']] = coords_p.shape[0]

In [15]:
# Mostrar las particulas resaltadas

#detections_img = np.uint8(img_out*255)
#for x, y in zip(particles['x'], particles['y']):
#    detections_img = cv2.circle(detections_img, (int(y), int(x)), 4, thickness=2, color=255)

import matplotlib.patches as mpatches
%matplotlib qt

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 12))
ax.imshow(img_out)

for index, row in particles.iterrows():
    x, y = row['x'], row['y']
    rect = mpatches.Rectangle((y-5, x-5), 10, 10, fill=False, edgecolor='red', linewidth=1)
    ax.add_patch(rect)


In [25]:
particles

,id,x,y,total_pixels
0,1,0,65,1
1,2,0.25,391,4
2,3,1,526.5,6
3,4,2,65,1
4,5,3.5,96,8
...,...,...,...,...
789,790,578.75,242,4
790,791,578.667,320.333,3
791,792,578.6,431.2,5
792,793,578.667,496.5,6


In [27]:
for index, row in particles.iterrows():
    df_out = df_out.append({'x': row['x'], 'y': row['y'], 'frame': nro_frame,
                                   'total_pixels': row['total_pixels']}, ignore_index=True) # rellenar dataframe
    print(row['x'], row['y'])

0.0 65.0
0.25 391.0
1.0 526.5
2.0 65.0
3.5 96.0
3.7142857142857144 119.0
3.5 218.5
3.8333333333333335 360.8333333333333
4.4 569.8
5.875 279.125
6.875 574.875
7.444444444444445 725.7777777777778
8.8 29.4
10.166666666666666 272.8333333333333
12.0 54.0
12.0 258.0
12.285714285714286 288.0
11.4 464.8
13.0 668.5
12.875 732.125
14.875 386.875
14.666666666666666 661.6666666666666
15.333333333333334 572.3333333333334
17.375 323.25
17.8 9.4
17.833333333333332 524.1666666666666
19.333333333333332 611.1111111111111
21.333333333333332 347.3333333333333
21.833333333333332 552.8333333333334
23.0 27.0
23.714285714285715 146.0
25.5 518.5
26.125 663.875
26.0 684.5
26.4 57.2
27.5 529.0
28.714285714285715 377.0
29.666666666666668 197.33333333333334
31.666666666666668 3.3333333333333335
31.875 105.875
31.8 131.0
32.5 437.0
33.5 55.5
33.666666666666664 648.3333333333334
36.0 601.5
35.5 654.0
38.125 333.875
38.5 192.0
39.57142857142857 265.2857142857143
40.0 366.0
39.5 564.5
39.5 572.0
39.875 762.875
41.1666

432.0 375.2857142857143
434.125 40.875
434.0 669.0
434.5 55.5
434.42857142857144 766.2857142857143
437.0 91.0
438.875 7.125
439.0 129.5
438.5 372.5
438.6666666666667 667.3333333333334
439.3333333333333 687.8888888888889
440.0 259.0
441.1666666666667 450.8333333333333
442.0 335.7142857142857
441.8 511.6
441.875 709.875
443.2 583.4
443.0 752.0
443.5 341.5
445.25 548.25
447.3333333333333 483.3333333333333
448.0 634.2857142857143
448.8 3.6
449.0 642.0
449.0 76.0
450.125 401.5
450.0 11.5
451.5 560.5
452.5 233.5
455.25 109.91666666666667
455.0 698.6666666666666
456.5 665.0
457.0 100.28571428571429
456.5 476.0
459.5 739.0
460.875 294.125
460.5 326.0
461.875 481.875
463.0 334.25
464.7142857142857 243.0
464.8333333333333 453.8333333333333
466.3333333333333 43.666666666666664
466.5 490.5
468.4 559.8
469.875 19.125
469.6 37.8
470.8 459.4
471.8 338.6
472.7142857142857 615.5714285714286
472.8333333333333 106.16666666666667
475.4 392.2
477.2142857142857 419.2142857142857
476.875 688.875
476.4 744.2
